In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableBranch
import os
from dotenv import load_dotenv

/Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
load_dotenv()

True

In [3]:
def tech_support_handler(request: str) -> str:
    print("\n-- Delegating to TECH SUPPORT handler --")
    # Simulate processing the technical issue
    return f"Tech Support handled request: '{request}' – [simulated technical assistance]"

def billing_support_handler(request: str) -> str:
    print("\n-- Delegating to BILLING SUPPORT handler --")
    # Simulate processing the billing/account issue
    return f"Billing Support handled request: '{request}' – [simulated billing resolution]"

def unclear_handler(request: str) -> str:
    print("\n-- Handling UNCLEAR request --")
    # Simulate asking for clarification
    return f"Sorry, I’m not sure how to categorize '{request}'. Could you clarify your request?"


In [4]:
from langchain_core.prompts import ChatPromptTemplate

router_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """You are an AI support assistant. 
     Analyze the user's request and decide which specialist should handle it:
     - If the request is about a technical problem or error, output "tech".
     - If the request is about billing, payments, or account issues, output "billing".
     - If the request is unclear or doesn't fit those categories, output "unclear".
     **Respond with only one word:** tech, billing, or unclear."""
    ),
    ("user", "{request}")
])


In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI  # or use ChatOpenAI for OpenAI models
from langchain_core.output_parsers import StrOutputParser

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)  # initialize a chat model

# Assemble the router chain:
router_chain = router_prompt | llm | StrOutputParser()

In [6]:
from langchain_core.runnables import RunnablePassthrough

branches = {
    "tech":    RunnablePassthrough.assign(output=lambda x: tech_support_handler(x['request']['request'])),
    "billing": RunnablePassthrough.assign(output=lambda x: billing_support_handler(x['request']['request'])),
    "unclear": RunnablePassthrough.assign(output=lambda x: unclear_handler(x['request']['request']))
}

In [7]:
from langchain_core.runnables import RunnableBranch

# Create the branching logic: check the decision and route accordingly
delegation_branch = RunnableBranch(
    (lambda x: x['decision'].strip() == "tech",    branches["tech"]),
    (lambda x: x['decision'].strip() == "billing", branches["billing"]),
    branches["unclear"]  # default branch if others don't match
)

In [8]:
# Combine router and branch into a single coordinated chain (the agent)
router_agent = {
    "decision": router_chain,             # produce a decision using the router LLM chain
    "request":  RunnablePassthrough()     # pass the original input through unchanged
} | delegation_branch | (lambda x: x['output'])

In [9]:
# Example 1: Technical support query
user_request1 = "The mobile app keeps crashing when I try to upload a photo."
result1 = router_agent.invoke({"request": user_request1})
print("Final result 1:", result1)

# Example 2: Billing support query
user_request2 = "I was charged twice on my credit card. How can I get a refund?"
result2 = router_agent.invoke({"request": user_request2})
print("Final result 2:", result2)

# Example 3: Unclear/general query
user_request3 = "Hello, I need some help with something."
result3 = router_agent.invoke({"request": user_request3})
print("Final result 3:", result3)



-- Delegating to TECH SUPPORT handler --
Final result 1: Tech Support handled request: 'The mobile app keeps crashing when I try to upload a photo.' – [simulated technical assistance]

-- Delegating to BILLING SUPPORT handler --
Final result 2: Billing Support handled request: 'I was charged twice on my credit card. How can I get a refund?' – [simulated billing resolution]

-- Handling UNCLEAR request --
Final result 3: Sorry, I’m not sure how to categorize 'Hello, I need some help with something.'. Could you clarify your request?


In [10]:
def booking_handler(request: str) -> str :
    print("\n -- DELEGATING TO BOOKING HANDLER    --")
    return f"""
                Booking Handler processed request:'{request}' Result: simulated booking action
            """

In [11]:
def info_handler(request: str) -> str:
    print(f"\n -- DELEGATING TO INFO HANDLER  -- \n")
    return f"""
            Info handler processed the request : '{request}' Result: Simulated information retrieval
    """

In [12]:
def unclear_handler(request: str) -> str:
    print("\n -- HANDLING UNCLER REQUEST --  \n")
    return f"""
            Coordinator could not delegate request : '{request}'. Please clarify
    """

In [15]:
coordinator_router_prompt = ChatPromptTemplate.from_messages([
                            ( "system", 
                                """
                                Analyze the user's request and determine which specialist handler should process it.   
                                - If the request is related to booking flights or hotels,      
                                output 'booker'.    - For all other general information questions, output 'info'.  
                                - If the request is unclear or doesn't fit either category,       output 'unclear'.     
                                ONLY output one word: 'booker', 'info', or 'unclear'.
                                """
                            ),
                            (
                                "user",
                                "{request}"
                                                                
                            )
                            
                                
                            ])

In [17]:
if llm:
    coordinator_router_chain = coordinator_router_prompt | llm | StrOutputParser()

In [25]:
branches = {
             "booker": RunnablePassthrough.assign(output=lambda x: booking_handler(x['request']['request'])),
             "info":    RunnablePassthrough.assign(output=lambda x: info_handler(x['request']['request'])),
             "unclear": RunnablePassthrough.assign(output=lambda x: unclear_handler(x['request']['request']))
           }


                                    

In [27]:
# Create the RunnableBranch. It takes the output of the router chain # and routes the original input ('request') to the corresponding handler. 

delegation_branch = RunnableBranch( (lambda x: x['decision'].strip() == 'booker', branches["booker"]), 
                                    (lambda x: x['decision'].strip() == 'info', branches["info"]), 
                                     branches["unclear"]
                                  )

In [29]:
coordinator_agent = {
                        "decision": coordinator_router_chain,
                        "request": RunnablePassthrough()

                    } | delegation_branch | (lambda x: x['output']) 

In [34]:
def main():
    if not llm:
        print("\n Skipping execution due to LLM inititalization failure")
        return
    print("\n --- Running with a booking request --")
    request_a = "Book me a flight to London"
    result_a = coordinator_agent.invoke({"request": request_a})
    print(f"Final Result A: {result_a}")

    print("\n --- Running with a info request --")
    request_b = "What is capital of Italy"
    result_b = coordinator_agent.invoke({"request": request_b})
    print(f"Final Result B: {result_b}")

    print("\n --- Running with a unclear request --")
    request_c = "Tell me about quantum physics"
    result_c = coordinator_agent.invoke({"request": request_c})
    print(f"Final Result C: {result_c}")

    
    

In [35]:
main()


 --- Running with a booking request --

 -- DELEGATING TO BOOKING HANDLER    --
Final Result A: 
                Booking Handler processed request:'Book me a flight to London' Result: simulated booking action
            

 --- Running with a info request --

 -- DELEGATING TO INFO HANDLER  -- 

Final Result B: 
            Info handler processed the request : 'What is capital of Italy' Result: Simulated information retrieval
    

 --- Running with a unclear request --

 -- DELEGATING TO INFO HANDLER  -- 

Final Result C: 
            Info handler processed the request : 'Tell me about quantum physics' Result: Simulated information retrieval
    
